In [ ]:
"""This program trains a binary classification model on image data (human face images with and without eyeglasses), 
saves the trained model, and classifies new/test images using the model. Also called binary image classification model.
It utilizes the Keras library and follows the typical structure of a Convolutional Neural Network (CNN) model. 
The trained model predicts/classifies the test/new images as: 0 - No Eye glasses & 1 - Eye glasses present.

This program is hosted online via Streamlit community cloud. 
Streamlit web app is created that allows users to upload an image, classify it using pre-trained models, and
display the result."""

In [ ]:
# Importing all necessary libraries
import h5py
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Setting up training and validation directories
train_data_dir = 'train'
validation_data_dir = 'test'

# Counting the number of images in training and validation directories
train_count = len(os.listdir('train/none')) + len(os.listdir('train/present'))
test_count = len(os.listdir('test/none')) + len(os.listdir('test/present'))

# Setting up image counts for training and validation
nb_train_samples = train_count
nb_validation_samples = test_count

# Setting up training parameters
epochs = 50    # specific for Model 2
batch_size = 100

# Setting image dimensions
img_width, img_height = 224, 224

In [ ]:
# Checking the image data format
if K.image_data_format() == 'channels_first':
	input_shape = (3, img_width, img_height)
else:
	input_shape = (img_width, img_height, 3)

In [ ]:
# Creating a Sequential model
model = Sequential()

# Adding Convolutional and Pooling layers
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening and adding Dense layers
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
# Compiling the model
model.compile(loss='binary_crossentropy',
			optimizer='rmsprop',
			metrics=['accuracy'])

In [ ]:
# Setting up data augmentation for training
train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

# Setting up data normalization for testing
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Creating data generators for training and validation
train_generator = train_datagen.flow_from_directory(train_data_dir,target_size=(img_width, img_height),
batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

# Training the model
history=model.fit_generator(
    train_generator,
	steps_per_epoch=nb_train_samples // batch_size,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=nb_validation_samples // batch_size)

In [ ]:
# Plotting loss and accuracy over epochs
plt.figure(figsize=(20,5))

# Plotting loss & validation loss
plt.subplot(1,2,1)
sns.lineplot(x=history.epoch, y=history.history['loss'], color='red', label='Train Loss')
sns.lineplot(x=history.epoch, y=history.history['val_loss'], color='orange', label='Val Loss')
plt.title('Loss on train vs test')
plt.legend(loc='best')

# Plotting accuracy and validation accuracy
plt.subplot(1,2,2)
sns.lineplot(x=history.epoch, y=history.history['accuracy'], color='blue', label='Train Accuracy')
sns.lineplot(x=history.epoch, y=history.history['val_accuracy'], color='green', label='Val Accuracy')
plt.title('Accuracy on train vs test')
plt.legend(loc='best')

plt.show()

In [ ]:
# Saving the trained model
model.save('Model3.h5') 

In [ ]:
# Generating Classification report
from sklearn.metrics import classification_report

# Setting up data augmentation for training
train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

# Setting up data normalization for testing
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Creating data generators for training and validation
train_generator = train_datagen.flow_from_directory(train_data_dir,target_size=(img_width, img_height),
batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

# Load the saved model
from keras.models import load_model
model = load_model('Model3.h5')

# Generate predictions for the validation dataset
validation_generator.reset()  # Reset generator to start from beginning
y_pred = model.predict_generator(validation_generator, steps=len(validation_generator), verbose=1)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

# Get true labels
y_true = validation_generator.classes

# Generate classification report
print(classification_report(y_true, y_pred_binary))

In [ ]:
# Loading the trained model for predictions
from keras.models import load_model
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import os
import numpy as np

model = load_model('Model3.h5')

In [ ]:
# Making predictions on test/new images using the trained model from the above.
import glob

# Setting the directory for testing images
folder_dir = "/Users/tabal/OneDrive/Desktop/Data Capstone/Pictures for Testing"

# Looping through each image in the directory
for image in glob.iglob(f'{folder_dir}/*'):
    
    # Loading and preprocessing the image
    load_image = load_img(image, target_size=(224, 224))
    img = img_to_array(load_image)
    img = preprocess_input(img.reshape(1,224,224,3))
    
    # Making predictions using the loaded model
    label = model.predict(img)
    
    # Displaying the predicted class (0 - No Eye glasses , 1 - Eye glasses present) for each image in the folder
    print("Predicted Class (0 - None , 1- Present) for ", os.path.basename(image), " is:", round(label[0][0]))

In [ ]:
# Making predictions on test/new images using the trained model from the above and exporting classification
# results to an Excel file.

import os
import glob

# Setting the directory for testing images
# folder_dir = "/Users/tabal/OneDrive/Desktop/Data Capstone/Pictures for Testing"
folder_dir = "/Users/tabal/Downloads/new_faces"

# Create an empty list to store prediction results
prediction_results = []

# Looping through each image in the directory
for image in glob.iglob(f'{folder_dir}/*'):
    
    # Loading and preprocessing the image
    load_image = load_img(image, target_size=(224, 224))
    img = img_to_array(load_image)
    img = preprocess_input(img.reshape(1,224,224,3))
    
    # Making predictions using the loaded model
    label = model.predict(img)
    
    # Append the prediction results to the list
    prediction_results.append({
        'Image Name': os.path.basename(image),
        'Prediction': round(label[0][0])})

# Create a DataFrame from the list
df = pd.DataFrame(prediction_results)

# Save the DataFrame to an Excel file
df.to_excel('classification_results_Model3.xlsx', index=False)